In [27]:
from transformers import AutoProcessor, BarkModel
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
processor = AutoProcessor.from_pretrained("suno/bark")

model = BarkModel.from_pretrained("suno/bark").to(device)
model = model.to_bettertransformer()
model.enable_cpu_offload()

/home/fenan/Builds/shikinha-2.0/venv/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/home/fenan/Builds/shikinha-2.0/venv/lib/python3.11/site-packages/transformers/models/encodec/modeling_encodec.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)
The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more detail

In [28]:
#txt = "Sim, você provavelmente está se referindo ao personagem Sonic the Hedgehog da série de jogos de vídeo game com o mesmo nome. Embora Sonic não seja exatamente descrito como um \"furry\" no sentido tradicional da subcultura furry, ele é um personagem antropomórfico azul que assemelha-se a um ouriço e coleta anéis dourados ao longo dos níveis nos jogos."
voice_preset = "v2/pt_speaker_5"

inputs = processor("Olá, bom dia! O meu nome é Shika, e eu sou o mais novo bot do cantinho.", voice_preset=voice_preset)
inputs = inputs.to(device)

audio_array = model.generate(**inputs)
audio_array = audio_array.cpu().numpy().squeeze()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


In [29]:
from IPython.display import Audio

sample_rate = model.generation_config.sample_rate
Audio(audio_array, rate=sample_rate)

In [26]:
import scipy

sample_rate = model.generation_config.sample_rate
scipy.io.wavfile.write("output.wav", rate=sample_rate, data=audio_array)